In [23]:
import duckdb

# connect to duckdb database
con = duckdb.connect("../data/analysis.duckdb")

In [24]:
df = con.execute("SELECT area, zoning_pressure " \
"FROM annual_metrics " \
"WHERE Year = '2023' " \
"ORDER BY Zoning_Pressure DESC;").df()
print(df)

                              Area  Zoning_pressure
0          Seattle-Tacoma-Bellevue         1.566971
1  Washington-Arlington-Alexandria         1.527613
2  Minneapolis-St Paul-Bloomington         1.469802
3     Austin-Round Rock-Georgetown         1.253099
4     Portland-Vancouver-Hillsboro         1.191338


In [25]:
df = con.execute("SELECT year, area, structural_gap, " \
"RANK() OVER (PARTITION BY year ORDER BY structural_gap DESC) " \
"as gap_rank " \
"FROM cumulative_metrics " \
"WHERE year > CAST(2020 AS VARCHAR) " \
"ORDER BY year, gap_rank " ).df()
print(df)

    Year                             Area  Structural_Gap  gap_rank
0   2021     Austin-Round Rock-Georgetown        1.000000         1
1   2021  Minneapolis-St Paul-Bloomington        1.000000         1
2   2021     Portland-Vancouver-Hillsboro        1.000000         1
3   2021          Seattle-Tacoma-Bellevue        1.000000         1
4   2021  Washington-Arlington-Alexandria        1.000000         1
5   2022     Austin-Round Rock-Georgetown        1.346774         1
6   2022     Portland-Vancouver-Hillsboro        1.264976         2
7   2022          Seattle-Tacoma-Bellevue        1.217445         3
8   2022  Minneapolis-St Paul-Bloomington        1.082444         4
9   2022  Washington-Arlington-Alexandria        0.880797         5
10  2023          Seattle-Tacoma-Bellevue        1.907701         1
11  2023     Austin-Round Rock-Georgetown        1.687641         2
12  2023  Minneapolis-St Paul-Bloomington        1.590979         3
13  2023     Portland-Vancouver-Hillsboro       

In [26]:
df = con.execute("SELECT area, AVG(zoning_pressure) AS avg_pressure " \
"FROM annual_metrics " \
"GROUP BY area " \
"ORDER BY avg_pressure DESC;").df()
print(df)

                              Area  avg_pressure
0          Seattle-Tacoma-Bellevue      1.184680
1     Austin-Round Rock-Georgetown      1.174240
2     Portland-Vancouver-Hillsboro      1.166250
3  Minneapolis-St Paul-Bloomington      1.160572
4  Washington-Arlington-Alexandria      1.116282


In [27]:
con.close()